In [1]:
import json

In [69]:
with open('../testcases/test1/NFA.json') as f:
    nfa = json.load(f)


terminating_states = set()
#loop through the states, and make each transition a list if it isn't already
for state in nfa:
    if state == 'startingState':
        nfa[state] = [nfa[state]]
        continue
    
    for transition in nfa[state]:
        if transition == 'isTerminatingState':
            if nfa[state][transition] == True:
                terminating_states.add(state)
            continue
        if type(nfa[state][transition]) != list:
            nfa[state][transition] = [nfa[state][transition]]

print(nfa)
print(terminating_states)

{'startingState': ['S3'], 'S3': {'isTerminatingState': False, 'ε': ['S1', 'S4']}, 'S4': {'isTerminatingState': False, 'ε': ['S7']}, 'S1': {'isTerminatingState': False, 'a': ['S2']}, 'S2': {'isTerminatingState': False, 'ε': ['S3', 'S4']}, 'S7': {'isTerminatingState': False, 'ε': ['S5', 'S8']}, 'S8': {'isTerminatingState': False, 'ε': ['S9']}, 'S5': {'isTerminatingState': False, 'b': ['S6']}, 'S6': {'isTerminatingState': False, 'ε': ['S7', 'S8']}, 'S9': {'isTerminatingState': False, 'c': ['S10']}, 'S10': {'isTerminatingState': False, 'ε': ['S11']}, 'S11': {'isTerminatingState': False, 'a': ['S12']}, 'S12': {'isTerminatingState': True}}
{'S12'}


## NFA 2 DFA

In [80]:
#global variable to store the closures of each state
#used to avoid recalculating the closure of a state
global_closuers = dict()

def get_possible_inputs(states):
    possible_transitions = set()
    # print(states)
    for state in states:
        for symbol in nfa[state]:
            if symbol != "ε" and symbol != "isTerminatingState":
                # print(symbol)
                possible_transitions.add(symbol)
    return possible_transitions

def epsilon_closure(nfa, state, prev_closure=set()):
    # print(state)
    if state in global_closuers:
        return global_closuers[state]


    #initialize the closure of the current state
    # with the current state
    closure = set()
    closure.add(state)
    
    # add epsilon transitions of the current state
    epsilon_transitions = set(nfa[state].get("ε", []))
    
    # print(epsilon_transitions)
    for s in epsilon_transitions:
        #to avoid infinite loop, check if the state is already in the closure
        if s not in prev_closure:
            #union of the current closure and the closure of the next state
            closure |= epsilon_closure(nfa, s, closure)

    global_closuers[state] = closure
    return closure

def move(nfa, states, symbol):
    result = set()
    for state in states:
        if symbol in nfa[state]:
            if symbol == "ε" or symbol == "isTerminatingState":
                continue
            #print(nfa[state][symbol])
            result |= set(nfa[state][symbol])
    return result

def nfa_to_dfa(nfa):
    dfa = {}
    #used to use as a key in dfa, sets are not hashable
    #because they are mutable, but frozensets are immutable
    starting_state = nfa["startingState"][0]
    start_state = frozenset(epsilon_closure(nfa, starting_state))
    is_terminating = False

    dfa[start_state] = {}
    queue = [start_state]
    while queue:
        current_state = queue.pop(0)
        if current_state & terminating_states:
            is_terminating = True
        dfa[current_state]["isTerminatingState"] = is_terminating
            
        possible_inputs = get_possible_inputs(current_state)
        for minput in possible_inputs:
            possible_transitions = move(nfa, current_state, minput)
            next_states_closures = set()
            for next_state in possible_transitions:
                next_states_closures |= epsilon_closure(nfa, next_state)
                
            closure = frozenset(next_states_closures)
            if closure not in dfa:
                dfa[closure] = {}
                queue.append(closure)
            dfa[current_state][minput] = next_states_closures
            
    return dfa

dfa = nfa_to_dfa(nfa)
print(dfa)

{frozenset({'S8', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}): {'isTerminatingState': False, 'a': {'S8', 'S2', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, frozenset({'S8', 'S2', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}): {'isTerminatingState': False, 'a': {'S8', 'S2', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, frozenset({'S8', 'S5', 'S7', 'S6', 'S9'}): {'isTerminatingState': False, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, frozenset({'S11', 'S10'}): {'isTerminatingState': False, 'a': {'S12'}}, frozenset({'S12'}): {'isTerminatingState': True}}


## Clean States

In [85]:
index = 0
dfa_cleaned = {}
for state in dfa:
    state_name = "S" + str(index)
    dfa_cleaned[state_name] = {}
    for transition in dfa[state]:
        dfa_cleaned[state_name][transition] = dfa[state][transition]
    index += 1

print(dfa_cleaned)

{'S0': {'isTerminatingState': False, 'a': {'S8', 'S2', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, 'S1': {'isTerminatingState': False, 'a': {'S8', 'S2', 'S5', 'S7', 'S4', 'S9', 'S3', 'S1'}, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, 'S2': {'isTerminatingState': False, 'b': {'S8', 'S5', 'S7', 'S6', 'S9'}, 'c': {'S11', 'S10'}}, 'S3': {'isTerminatingState': False, 'a': {'S12'}}, 'S4': {'isTerminatingState': True}}


## Minimize DFA

In [86]:
def minimize_dfa(dfa):
    #initialize the partition
    partition = [set(), set()]
    for state in dfa:
        if dfa[state]["isTerminatingState"]:
            partition[0].add(state)
        else:
            partition[1].add(state)
    print(partition)



[{'S4'}, {'S0', 'S2', 'S3', 'S1'}]
None
